## Test prior extend value

In [1]:
import numpy as np
import pandas as pd
from matplotlib import cm
import matplotlib.pyplot as plt
import os

RANDOM_SEED = 1

In [2]:
full = pd.read_csv('test_full.csv')
test = pd.read_csv('test_selected.csv')
val = pd.read_csv('val_selected.csv')
train = pd.read_csv('train_selected.csv')

test['split'] = 'test'
val['split'] = 'xval'
train['split'] = 'train'

total = pd.concat([test, val, train], axis=0).sort_values('patient').drop_duplicates(subset=['scan_id'], keep='first')
total = total
total[['patient', 'scan_id', 'split']].reset_index(drop=True).to_latex('grouped_split.tex')


In [3]:
total.groupby(['split','source']).count().reset_index().pivot(index='source', columns='split', values='img').to_latex('summary_split.tex')

In [4]:
import sys
sys.path.append(os.path.abspath('../src/datasets/'))

from StratifiedGroupKFold import StratifiedGroupKFold

ModuleNotFoundError: No module named 'StratifiedGroupKFold'

In [ ]:
USB_PATH = '/media/jan/JAL/data/'
test_file = os.path.join(USB_PATH, 'test_full.csv')

In [ ]:
data_df = pd.read_csv(test_file, index_col=0)
data_df.head()

,img,tgt,scan_id,slice_id,patient,source
0,/root/space/output/dataset_2_contrast_3/xVertS...,/root/space/output/dataset_2_contrast_3/xVertS...,xVertSeg_004,9,xVertSeg_001,xVertSeg
1,/root/space/output/dataset_2_contrast_3/xVertS...,/root/space/output/dataset_2_contrast_3/xVertS...,xVertSeg_004,397,xVertSeg_001,xVertSeg
2,/root/space/output/dataset_2_contrast_3/xVertS...,/root/space/output/dataset_2_contrast_3/xVertS...,xVertSeg_004,192,xVertSeg_001,xVertSeg
3,/root/space/output/dataset_2_contrast_3/xVertS...,/root/space/output/dataset_2_contrast_3/xVertS...,xVertSeg_004,183,xVertSeg_001,xVertSeg
4,/root/space/output/dataset_2_contrast_3/xVertS...,/root/space/output/dataset_2_contrast_3/xVertS...,xVertSeg_004,355,xVertSeg_001,xVertSeg


In [ ]:
data_df.groupby(['source']).slice_id.count()

source
MyoSegmenTUM    4754
USiegen         1154
xVertSeg        6265
Name: slice_id, dtype: int64

In [ ]:
data_df.groupby(['scan_id']).slice_id.count()


scan_id
MyoSegmenTUM_001     81
MyoSegmenTUM_002     81
MyoSegmenTUM_003     81
MyoSegmenTUM_004     81
MyoSegmenTUM_005     80
                   ... 
xVertSeg_011        319
xVertSeg_012        314
xVertSeg_013        338
xVertSeg_014        558
xVertSeg_015        404
Name: slice_id, Length: 86, dtype: int64

In [ ]:
dev_test_split = StratifiedGroupKFold(n_splits=6, random_state=RANDOM_SEED, shuffle=True)
train_val_split = StratifiedGroupKFold(n_splits=5, random_state=RANDOM_SEED, shuffle=True)

ix_dev, ix_test = next(dev_test_split.split(X = data_df.slice_id, y = data_df.source, groups = data_df.scan_id ))

dev_df, test_df = data_df.iloc[ix_dev], data_df.iloc[ix_test]
ix_train, ix_val = next(train_val_split.split(X = dev_df.slice_id, y = dev_df.source, groups = dev_df.scan_id ))
train_df, val_df = dev_df.iloc[ix_train], dev_df.iloc[ix_val]

In [ ]:
counts = pd.concat([df.groupby(['source']).scan_id.count().rename(name) for df, name in zip([train_df, test_df, val_df], ['train', 'test', 'val'])], axis= 1)

In [ ]:
counts

,train,test,val
source,,,
MyoSegmenTUM,3193,781,780
USiegen,795,163,196
xVertSeg,4446,896,923


In [ ]:
data_df.drop_duplicates('patient', keep='first').iloc[-1]

img         /root/space/output/dataset_2_contrast_3/MyoSeg...
tgt         /root/space/output/dataset_2_contrast_3/MyoSeg...
scan_id                                      MyoSegmenTUM_025
slice_id                                                    9
patient                                      MyoSegmenTUM_054
source                                           MyoSegmenTUM
Name: 12093, dtype: object

In [ ]:
data_df.drop_duplicates('patient', keep='first').img.iloc[-1]

'/root/space/output/dataset_2_contrast_3/MyoSegmenTUM_images/image025/slice_009.npy'